In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [5]:
import pandas as pd

In [6]:
col_spec = [(0, 6), (7, 12), (13, 41), (42, 46), (47, 50), (51, 55), (56, 63), (64, 73), (73, 80), (81, 89), (90, 97)]

In [7]:
import os
import shutil

In [8]:
from os import walk

In [19]:
'''
data_dict = {}
data_dict['lat'] = []
data_dict['lon'] = []

for dirpath, dirnames, filenames in os.walk('data2019/'):
#     print(os.path.join("/", "USA_station_data"))
    for filename in filenames:
        usaf_id_this_file = filename[:6]
        wban_id_this_file = filename[7:12]

        if usaf_id_this_file in usaf_id_list and wban_id_this_file in wban_id_list:
            print(usaf_id_this_file)
            print('done')
            temp = data_usa[data_usa['USAF'] == usaf_id_this_file]
            lat = temp['LAT']
            lon = temp['LON']
            
            data_dict['lat'].append(lat)
            data_dict['lon'].append(lon)
            
            
            print(lat, lon)
            source = os.path.join(dirpath, filename)
            print(source)
            dest = "./USA_station_data"
            shutil.copy(source, dest)
'''

'\ndata_dict = {}\ndata_dict[\'lat\'] = []\ndata_dict[\'lon\'] = []\n\nfor dirpath, dirnames, filenames in os.walk(\'data2019/\'):\n#     print(os.path.join("/", "USA_station_data"))\n    for filename in filenames:\n        usaf_id_this_file = filename[:6]\n        wban_id_this_file = filename[7:12]\n\n        if usaf_id_this_file in usaf_id_list and wban_id_this_file in wban_id_list:\n            print(usaf_id_this_file)\n            print(\'done\')\n            temp = data_usa[data_usa[\'USAF\'] == usaf_id_this_file]\n            lat = temp[\'LAT\']\n            lon = temp[\'LON\']\n            \n            data_dict[\'lat\'].append(lat)\n            data_dict[\'lon\'].append(lon)\n            \n            \n            print(lat, lon)\n            source = os.path.join(dirpath, filename)\n            print(source)\n            dest = "./USA_station_data"\n            shutil.copy(source, dest)\n'

In [10]:
col_spec2 = [(0, 6), (7, 13), (14, 25), (26, 36), (37,-1)]

for dirpath, dirnames, filenames in os.walk("USA_station_data/"):
    for filename in filenames:
        data_df = pd.read_fwf(os.path.join(dirpath, filename))
        print(data_df.shape)

In [11]:
id_coordinates = pd.read_csv('id_world_coord.csv')

In [12]:
id_coordinates

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,7018,99999,WXPOD 7018,NaN,NaN,NaN,0.000,0.000,7018.0,20110309,20130730
1,7026,99999,WXPOD 7026,AF,NaN,NaN,0.000,0.000,7026.0,20120713,20170822
2,7070,99999,WXPOD 7070,AF,NaN,NaN,0.000,0.000,7070.0,20140923,20150926
3,8260,99999,WXPOD8270,NaN,NaN,NaN,0.000,0.000,0.0,20050101,20100920
4,8268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,20100519,20120323
...,...,...,...,...,...,...,...,...,...,...,...
29703,999999,96405,CORDOVA 14 ESE,US,AK,NaN,60.473,-145.354,25.3,20170727,20200531
29704,999999,96406,RUBY 44 ESE,US,AK,NaN,64.502,-154.130,78.9,20140828,20200531
29705,999999,96407,SELAWIK 28 E,US,AK,NaN,66.562,-159.004,6.7,20150813,20200531
29706,999999,96408,DENALI 27 N,US,AK,NaN,63.452,-150.875,678.2,20150819,20200531


In [13]:
usaf_id_list = set(id_coordinates['USAF'])
wban_id_list = set(id_coordinates['WBAN'])

In [14]:
col_spec2 = [(0, 6), (7, 13), (14, 25), (26, 36), (37, -1)]

dataframe_created = False
index = 1
for dirpath, dirnames, filenames in os.walk('data2019/'):
    for filename in filenames:
        usaf_id_this_file = int(filename[:6])
        wban_id_this_file = int(filename[7:12])
        print(usaf_id_this_file, wban_id_this_file)

        temp = id_coordinates[id_coordinates['USAF'] == usaf_id_this_file]

        temp = temp[temp['WBAN'] == wban_id_this_file]
        
        file = os.path.join(dirpath, filename)
        temperature_data = pd.read_fwf(file)
        temperature_data = temperature_data.filter(['YEARMODA', 'TEMP'])
        if temperature_data.empty or temp.empty:
            continue
        if not dataframe_created:
            cols = ['LAT', 'LON'] + list(temperature_data['YEARMODA'])
            print(cols)
            df = pd.DataFrame(columns=cols)
            dataframe_created = True
            
        data_dict = {}
        
        data_dict['LAT'] = temp.iloc[0]['LAT']
        data_dict['LON'] = temp.iloc[0]['LON']
        data_dict.update(dict(zip(temperature_data.YEARMODA, temperature_data.TEMP)))
#         print(data_dict)
        
        df.loc[index] = pd.Series(data_dict)
        index += 1
        
print(df)
df.to_csv("myRes.csv")

10010 99999
['LAT', 'LON', 20190101, 20190102, 20190103, 20190104, 20190105, 20190106, 20190107, 20190108, 20190109, 20190110, 20190111, 20190112, 20190113, 20190114, 20190115, 20190116, 20190117, 20190118, 20190119, 20190120, 20190121, 20190122, 20190123, 20190124, 20190125, 20190126, 20190127, 20190128, 20190129, 20190130, 20190131, 20190201, 20190202, 20190203, 20190204, 20190205, 20190206, 20190207, 20190208, 20190209, 20190210, 20190211, 20190212, 20190213, 20190214, 20190215, 20190216, 20190217, 20190218, 20190219, 20190220, 20190221, 20190222, 20190223, 20190224, 20190225, 20190226, 20190227, 20190228, 20190301, 20190302, 20190303, 20190304, 20190305, 20190306, 20190307, 20190308, 20190309, 20190310, 20190311, 20190312, 20190313, 20190314, 20190315, 20190316, 20190317, 20190318, 20190319, 20190320, 20190321, 20190322, 20190323, 20190324, 20190325, 20190326, 20190327, 20190328, 20190329, 20190330, 20190331, 20190401, 20190402, 20190403, 20190404, 20190405, 20190406, 20190407, 201

24520 99999
24530 99999
24560 99999
24580 99999
24600 99999
24620 99999
24640 99999
24680 99999
24690 99999
24760 99999
24810 99999
24820 99999
24840 99999
24853 99999
24860 99999
24870 99999
24880 99999
24890 99999
24900 99999
24930 99999
24960 99999
24980 99999
25000 99999
25010 99999
25050 99999
25103 99999
25130 99999
25140 99999
25160 99999
25180 99999
25200 99999
25260 99999
25350 99999
25360 99999
25390 99999
25400 99999
25420 99999
25440 99999
25450 99999
25460 99999
25480 99999
25490 99999
25500 99999
25520 99999
25540 99999
25560 99999
25570 99999
25580 99999
25590 99999
25610 99999
25620 99999
25625 99999
25630 99999
25635 99999
25650 99999
25670 99999
25710 99999
25740 99999
25750 99999
25870 99999
25890 99999
25900 99999
25980 99999
26040 99999
26050 99999
26110 99999
26150 99999
26160 99999
26180 99999
26220 99999
26230 99999
26250 99999
26280 99999
26290 99999
26320 99999
26350 99999
26360 99999
26410 99999
26440 99999
26480 99999
26510 99999
26610 99999
26640 99999
2667

63440 99999
63480 99999
63500 99999
63560 99999
63700 99999
63750 99999
63770 99999
63800 99999
63910 99999
64000 99999
64070 99999
64140 99999
64180 99999
64280 99999
64320 99999
64325 99999
64340 99999
64390 99999
64470 99999
64490 99999
64500 99999
64510 99999
64550 99999
64560 99999
64580 99999
64590 99999
64640 99999
64650 99999
64720 99999
64760 99999
64765 99999
64770 99999
64780 99999
64790 99999
64840 99999
64900 99999
64901 99999
64940 99999
64960 99999
65900 99999
66010 99999
66020 99999
66040 99999
66045 99999
66050 99999
66060 99999
66080 99999
66090 99999
66099 99999
66100 99999
66120 99999
66130 99999
66140 99999
66160 99999
66170 99999
66180 99999
66190 99999
66200 99999
66210 99999
66230 99999
66240 99999
66250 99999
66260 99999
66270 99999
66280 99999
66303 99999
66304 99999
66310 99999
66320 99999
66330 99999
66350 99999
66360 99999
66370 99999
66380 99999
66390 99999
66400 99999
66410 99999
66430 99999
66440 99999
66450 99999
66460 99999
66470 99999
66480 99999
6650

112780 99999
112800 99999
112850 99999
112900 99999
112920 99999
112960 99999
112980 99999
113010 99999
113020 99999
113040 99999
113080 99999
113100 99999
113120 99999
113160 99999
113180 99999
113250 99999
113300 99999
113310 99999
113320 99999
113370 99999
113400 99999
113430 99999
113450 99999
113460 99999
113480 99999
113490 99999
113500 99999
113510 99999
113550 99999
113560 99999
113570 99999
113580 99999
113590 99999
113620 99999
113650 99999
113660 99999
113670 99999
113750 99999
113780 99999
113800 99999
113820 99999
113830 99999
113840 99999
113850 99999
113860 99999
113870 99999
113890 99999
113900 99999
113930 99999
113950 99999
114060 99999
114140 99999
114230 99999
114232 99999
114233 99999
114380 99999
114480 99999
114500 99999
114570 99999
114640 99999
114870 99999
115020 99999
115090 99999
115180 99999
115190 99999
115200 99999
115351 99999
115380 99999
115460 99999
115670 99999
116030 99999
116240 99999
116280 99999
116360 99999
116430 99999
116520 99999
116530 99999

161720 99999
161790 99999
161810 99999
161900 99999
161910 99999
161920 99999
161970 99999
162040 99999
162060 99999
162140 99999
162150 99999
162160 99999
162180 99999
162190 99999
162210 99999
162240 99999
162300 99999
162310 99999
162320 99999
162340 99999
162350 99999
162390 99999
162420 99999
162430 99999
162440 99999
162450 99999
162520 99999
162530 99999
162580 99999
162591 99999
162592 99999
162593 99999
162600 99999
162610 99999
162630 99999
162660 99999
162700 99999
162710 99999
162800 99999
162880 99999
162890 34113
162923 99999
162940 99999
163100 99999
163120 99999
163140 99999
163160 99999
163170 99999
163180 99999
163200 99999
163240 99999
163250 99999
163320 99999
163340 99999
163370 99999
163440 99999
163445 99999
163500 99999
163600 99999
163620 99999
163640 99999
163650 99999
164000 99999
164050 99999
164160 99999
164200 99999
164220 99999
164290 99999
164340 99999
164360 99999
164420 99999
164500 99999
164530 99999
164590 99999
164600 99999
164640 99999
164650 99999

...

818470 99999
818480 99999
818600 99999
818630 99999
818660 99999
818680 99999
818690 99999
818700 99999
818720 99999
818730 99999
818750 99999
818760 99999
818770 99999
818780 99999
818880 99999
818900 99999
818930 99999
819000 99999
819010 99999
819030 99999
819050 99999
819080 99999
819090 99999
819100 99999
819110 99999
819120 99999
819130 99999
819140 99999
819150 99999
819160 99999
819170 99999
819180 99999
819210 99999
819220 99999
819240 99999
819270 99999
819320 99999
819400 99999
819410 99999
819500 99999
819510 99999
819530 99999
819540 99999
819550 99999
819560 99999
819570 99999
819580 99999
819630 99999
819650 99999
819700 99999
819750 99999
819760 99999
819770 99999
819780 99999
819790 99999
819820 99999
819850 99999
819860 99999
819870 99999
819880 99999
819890 99999
819900 99999
819910 99999
819920 99999
819940 99999
819950 99999
819960 99999
819970 99999
819980 99999
820170 99999
820220 99999
820305 99999
820306 99999
820910 99999
820980 99999
821070 99999
821080 99999

869550 99999
869560 99999
869570 99999
869580 99999
869600 99999
869610 99999
869620 99999
869630 99999
869640 99999
869650 99999
869660 99999
869670 99999
869680 99999
869690 99999
869700 99999
869710 99999
869720 99999
869730 99999
869750 99999
869760 99999
869770 99999
869780 99999
869790 99999
869800 99999
869810 99999
869820 99999
869830 99999
869840 99999
869850 99999
869860 99999
869880 99999
869890 99999
869900 99999
869920 99999
869930 99999
869940 99999
869950 99999
869960 99999
869980 99999
870070 99999
870160 99999
870220 99999
870221 99999
870460 99999
870470 99999
870650 99999
870780 99999
870970 99999
871210 99999
871271 99999
871290 99999
871480 99999
871550 99999
871620 99999
871660 99999
871780 99999
872170 99999
872220 99999
872440 99999
872570 99999
872700 99999
872890 99999
873110 99999
873200 99999
873280 99999
873440 99999
873470 99999
873490 99999
873710 99999
873740 99999
873930 99999
873950 99999
874050 99999
874160 99999
874180 99999
874200 99999
874360 99999

947860 99999
947910 99999
948020 99999
948040 99999
948070 99999
948080 99999
948090 99999
948110 99999
948120 99999
948130 99999
948140 99999
948170 99999
948200 99999
948210 99999
948220 99999
948260 99999
948270 99999
948280 99999
948290 99999
948300 99999
948330 99999
948350 99999
948360 99999
948370 99999
948380 99999
948390 99999
948400 99999
948420 99999
948430 99999
948460 99999
948500 99999
948520 99999
948540 99999
948550 99999
948570 99999
948590 99999
948600 99999
948620 99999
948630 99999
948640 99999
948650 99999
948660 99999
948700 99999
948720 99999
948740 99999
948750 99999
948780 99999
948810 99999
948860 99999
948890 99999
948910 99999
948920 99999
948930 99999
948940 99999
948955 99999
948960 99999
948980 99999
949030 99999
949050 99999
949060 99999
949070 99999
949080 99999
949100 99999
949110 99999
949120 99999
949130 99999
949140 99999
949150 99999
949190 99999
949210 99999
949250 99999
949260 99999
949270 99999
949290 99999
949300 99999
949330 99999
949340 99999

997380 99999
997381 99999
997382 99999
997383 99999
997384 99999
997386 99999
997387 99999
997687 99999
997688 99999
997691 99999
997692 99999
997693 99999
997694 99999
997695 99999
997696 99999
997697 99999
997698 99999
997699 99999
997701 99999
997703 99999
997704 99999
997705 99999
997707 99999
997708 99999
997729 99999
997731 99999
997732 99999
997733 99999
997734 99999
997735 99999
997736 99999
997738 99999
997739 99999
997741 99999
997742 99999
997743 99999
997779 99999
997781 99999
997782 99999
997784 99999
997785 99999
997787 99999
997788 99999
997789 99999
997792 99999
997793 99999
997796 99999
997797 99999
997798 99999
997799 99999
997802 99999
997804 99999
997982 99999
997988 99999
997989 99999
997990 99999
997991 99999
997993 99999
997994 99999
997996 99999
997997 99999
997999 99999
998000 99999
998001 99999
998002 99999
998003 99999
998004 99999
998005 99999
998007 99999
998008 99999
998009 99999
998010 99999
998011 99999
998012 99999
998013 99999
998014 99999
998015 99999

PermissionError: [Errno 13] Permission denied: 'myRes.csv'

In [15]:
def getStations(topLeftLat, topLeftLong, bottomRightLat, bottomRightLong, data=df):
    result_df = data[data['LAT'] <= topLeftLat]
    result_df = result_df[result_df['LAT'] >= bottomRightLat]
    result_df = result_df[result_df['LON'] <= topLeftLong]
    result_df = result_df[result_df['LON'] >= bottomRightLong]
    return result_df

In [16]:
subset_test = getStations(80.06, 16.06, 59.03, 5.34)

In [18]:
subset_test

,LAT,LON,20190101,20190102,20190103,20190104,20190105,20190106,20190107,20190108,...,20191222,20191223,20191224,20191225,20191226,20191227,20191228,20191229,20191230,20191231
2,59.792,5.341,39.7,36.4,36.5,45.6,NaN,42.5,38.9,40.6,...,45.7,42.6,NaN,NaN,35.6,38.7,NaN,46.4,43.5,NaN
4,77.000,15.500,3.9,11.6,27.8,23.6,27.9,29.0,25.6,24.9,...,20.0,22.7,23.9,18.6,15.7,25.3,24.0,20.1,12.2,6.1
6,78.917,11.933,3.6,8.2,20.6,27.5,24.1,24.9,21.3,21.7,...,16.3,20.0,18.7,14.4,12.8,21.9,20.0,15.3,8.0,6.1
7,78.246,15.466,4.4,7.8,20.9,21.3,25.9,23.2,22.0,20.2,...,18.6,21.0,20.8,16.5,10.1,21.5,19.5,16.1,6.9,1.9
13,77.683,14.783,5.3,11.1,25.8,23.0,27.9,26.7,23.5,23.8,...,16.0,18.5,20.4,18.3,9.4,21.9,17.9,13.7,4.9,-4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,60.422,15.515,36.3,28.1,29.9,38.5,35.1,35.3,33.3,31.3,...,33.9,29.5,29.5,31.8,27.7,24.2,23.3,39.8,44.0,36.6
375,60.967,14.500,35.2,27.8,29.6,39.4,33.1,30.6,28.5,30.4,...,32.6,25.8,22.4,28.5,26.9,20.3,22.5,39.9,41.5,35.8
380,59.300,14.900,36.9,28.6,29.3,33.4,34.0,32.5,32.4,30.5,...,38.4,35.2,34.9,33.9,30.6,25.2,24.1,37.3,43.5,35.2
387,60.383,15.133,30.8,22.6,26.2,34.5,33.1,34.6,30.8,26.8,...,31.5,29.5,30.4,28.9,24.5,18.8,20.7,35.1,37.2,30.7
